# 🏥 통합 영양 중재 시뮬레이터 (완전판)
## ML 모델 (Albumin) + Rule-Based 모델 통합

---

### 📌 개요
- **알부민**: MIMIC-IV 기반 ML 모델 (Random Forest/XGBoost)
- **기타 영양소**: 문헌 기반 Rule-based 모델
  - ✅ Iron (Hemoglobin)
  - ✅ Vitamin D
  - ✅ Calcium (Fracture Risk)
  - ✅ Omega-3 (CVD Risk)
  - ✅ **Vitamin C** (새로 추가!)

### ⚠️ 중요: 라이브러리 설치 후 **런타임 재시작** 필수!

## 1️⃣ 환경 설정

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 라이브러리 설치
# ⚠️ 이 셀 실행 후 "런타임 > 런타임 다시 시작" 필수!

!pip install --upgrade scikit-learn xgboost joblib -q

print("""\n
✅ 설치 완료!

⚠️  다음 단계:
1. 상단 메뉴 "런타임" → "런타임 다시 시작" 클릭
2. 이 셀은 건너뛰고 다음 셀부터 순서대로 실행
""")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 MB 7.3 MB/s eta 0:00:00


✅ 설치 완료!

⚠️  다음 단계:
1. 상단 메뉴 "런타임" → "런타임 다시 시작" 클릭
2. 이 셀은 건너뛰고 다음 셀부터 순서대로 실행



## 2️⃣ 라이브러리 임포트 (런타임 재시작 후 여기서부터)

In [ ]:
# Drive 재마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
import joblib
import numpy as np
import pandas as pd
from typing import Dict, Any, Optional, List
from dataclasses import dataclass, field

warnings.filterwarnings('ignore')

# 버전 확인
import sklearn, scipy
print("✅ 라이브러리 로드 완료")
print(f"   scikit-learn: {sklearn.__version__}")
print(f"   numpy: {np.__version__}")
print(f"   scipy: {scipy.__version__}")

✅ 라이브러리 로드 완료
   scikit-learn: 1.8.0
   numpy: 2.0.2
   scipy: 1.16.3


## 3️⃣ 데이터 클래스

In [ ]:
@dataclass
class PatientProfile:
    """환자 프로필"""
    age: int
    sex: str  # 'M' or 'F'

    # Lab values
    hemoglobin: Optional[float] = None
    ferritin: Optional[float] = None
    tsat: Optional[float] = None
    albumin: Optional[float] = None
    vitamin_d: Optional[float] = None
    calcium: Optional[float] = None
    crp: Optional[float] = None

    # ML 모델용
    bun: Optional[float] = None
    creatinine: Optional[float] = None
    glucose: Optional[float] = None
    sodium: Optional[float] = None
    potassium: Optional[float] = None
    chloride: Optional[float] = None
    bicarbonate: Optional[float] = None
    wbc: Optional[float] = None
    platelet: Optional[float] = None

    # Clinical
    ckd_stage: int = 0
    smoker: bool = False
    immune_compromised: bool = False
    chronic_inflammation: bool = False
    kidney_stone_history: bool = False
    hemochromatosis: bool = False
    hypercalcemia: bool = False
    fracture_risk_high: bool = False

@dataclass
class NutritionIntervention:
    """영양 중재 계획"""
    iron_mg: Optional[float] = None
    vitamin_d_iu: Optional[float] = None
    calcium_mg: Optional[float] = None
    omega3_epa_dha_g: Optional[float] = None
    vitamin_c_mg: Optional[float] = None  # ✅ Vitamin C
    protein_g: Optional[float] = None
    duration_weeks: int = 4

@dataclass
class SimulationResult:
    """시뮬레이션 결과"""
    parameter: str
    current_value: Optional[float]
    expected_value: Optional[float]
    expected_change: Optional[float]
    interpretation: str
    warnings: List[str] = field(default_factory=list)
    contraindications: List[str] = field(default_factory=list)
    monitoring_recommendations: List[str] = field(default_factory=list)
    model_type: str = "rule-based"

print("✅ 데이터 클래스 정의 완료")

✅ 데이터 클래스 정의 완료


## 4️⃣ ML 모델 로더

In [ ]:
class AlbuminMLPredictor:
    """알부민 ML 모델"""

    def __init__(self, model_path: str):
        try:
            self.model_data = joblib.load(model_path)
            self.model = self.model_data['model']
            self.feature_names = self.model_data['feature_names']
            self.model_type = self.model_data.get('model_type', 'Unknown')
            self.test_r2 = self.model_data.get('test_r2', None)

            print(f"✅ ML 모델 로드: {self.model_type}")
            if self.test_r2:
                print(f"   Test R²: {self.test_r2:.4f}")
        except Exception as e:
            raise ValueError(f"모델 로드 실패: {e}")

    def predict(self, patient_data: Dict[str, float]) -> float:
        X = [patient_data[f] for f in self.feature_names]
        return float(self.model.predict(np.array(X).reshape(1, -1))[0])

    def get_required_features(self) -> List[str]:
        return self.feature_names.copy()

print("✅ AlbuminMLPredictor 정의 완료")

✅ AlbuminMLPredictor 정의 완료


## 5️⃣ 통합 시뮬레이터 (Vitamin C 포함)

In [ ]:
class IntegratedNutritionSimulator:
    """통합 영양 중재 시뮬레이터"""

    def __init__(self, albumin_ml_model: Optional[AlbuminMLPredictor] = None):
        self.albumin_ml_model = albumin_ml_model
        self._load_guidelines()

        if albumin_ml_model:
            print("🤖 ML + Rule-based 통합 모드")
        else:
            print("📋 Rule-based 전용 모드")

    def _load_guidelines(self):
        """가이드라인 로드"""
        self.guidelines = {
            'iron': {
                'baseline_hgb_increase': 1.0,
                'ckd_absorption_factor': 0.7,
                'inflammation_factor': 0.7,
            },
            'vitamin_d': {
                'increase_per_1000iu': 7.0,
                'upper_limit_iu': 4000,
            },
            'calcium': {
                'fracture_risk_reduction': 0.15,
            },
            'omega3': {
                'cvd_risk_reduction': 0.08,
            },
            'vitamin_c': {
                'rni': 100,
                'optimal_range': (200, 500),
                'upper_limit': 2000,
                'kidney_stone_risk_threshold_male': 1000,
            }
        }

    def simulate(self, patient: PatientProfile, intervention: NutritionIntervention):
        """시뮬레이션 실행"""
        results = {}

        if intervention.protein_g and self.albumin_ml_model:
            results['albumin'] = self._simulate_albumin_ml(patient, intervention)

        if intervention.iron_mg:
            results['hemoglobin'] = self._simulate_iron(patient, intervention)

        if intervention.vitamin_d_iu:
            results['vitamin_d'] = self._simulate_vitamin_d(patient, intervention)

        if intervention.calcium_mg:
            results['fracture_risk'] = self._simulate_calcium(patient, intervention)

        if intervention.omega3_epa_dha_g:
            results['cvd_risk'] = self._simulate_omega3(patient, intervention)

        if intervention.vitamin_c_mg:
            results['vitamin_c'] = self._simulate_vitamin_c(patient, intervention)

        return results

    def _prepare_patient_features(self, patient: PatientProfile):
        """ML feature 준비"""
        defaults = {
            'hemoglobin': 13.0, 'bun': 20.0, 'creatinine': 1.0,
            'glucose': 100.0, 'sodium': 140.0, 'potassium': 4.0,
            'chloride': 105.0, 'bicarbonate': 24.0,
            'wbc': 8.0, 'platelet': 250.0,
        }

        features = {'age': patient.age, 'sex_M': 1 if patient.sex == 'M' else 0}
        for k, v in defaults.items():
            features[k] = getattr(patient, k, None) or v

        features['bun_creatinine_ratio'] = features['bun'] / max(features['creatinine'], 0.1)
        return features

    def _simulate_albumin_ml(self, patient, intervention):
        """알부민 ML 예측"""
        current = patient.albumin

        try:
            feats = self._prepare_patient_features(patient)
            predicted = self.albumin_ml_model.predict(feats)

            protein_effect = min(intervention.protein_g / 50.0, 1.5)
            duration_factor = min(intervention.duration_weeks / 8.0, 1.0)

            adjustment = (predicted - (current or 3.5)) * protein_effect * duration_factor
            expected = (current or 3.5) + adjustment

            interp = f"ML 예측: 단백질 {intervention.protein_g}g/day, {intervention.duration_weeks}주"
            if adjustment > 0:
                interp += f" → +{adjustment:.2f} g/dL"

            warnings = []
            if patient.ckd_stage >= 3:
                warnings.append("CKD: 고단백 주의")

            return SimulationResult(
                parameter="Albumin",
                current_value=current,
                expected_value=expected,
                expected_change=adjustment,
                interpretation=interp,
                model_type="ml",
                warnings=warnings,
                monitoring_recommendations=["4주 후 알부민 재검사", "신기능 모니터링"]
            )
        except Exception as e:
            return SimulationResult(
                parameter="Albumin",
                current_value=current,
                expected_value=None,
                expected_change=None,
                interpretation=f"ML 예측 실패: {e}",
                model_type="ml"
            )

    def _simulate_iron(self, patient, intervention):
        """철분 → Hemoglobin"""
        current = patient.hemoglobin
        dose = intervention.iron_mg
        weeks = intervention.duration_weeks

        increase = self.guidelines['iron']['baseline_hgb_increase']
        factor = 1.0
        warnings = []

        if patient.ckd_stage >= 3:
            factor *= 0.7
            warnings.append("CKD: 흡수율 ↓30%")
        if patient.chronic_inflammation or (patient.crp and patient.crp > 5):
            factor *= 0.7
            warnings.append("염증: 효과 감소")

        expected_change = increase * factor * (dose/100) * (weeks/4)
        expected = (current + expected_change) if current else None

        return SimulationResult(
            parameter="Hemoglobin",
            current_value=current,
            expected_value=expected,
            expected_change=expected_change,
            interpretation=f"철분 {dose}mg/day, {weeks}주 → Hgb +{expected_change:.1f} g/dL",
            warnings=warnings,
            monitoring_recommendations=["4주 후 CBC", "Ferritin/TSAT 추적"]
        )

    def _simulate_vitamin_d(self, patient, intervention):
        """비타민 D"""
        current = patient.vitamin_d
        dose = intervention.vitamin_d_iu
        weeks = intervention.duration_weeks

        time_factor = min(weeks / 12.0, 1.0)
        increase = (dose / 1000.0) * 7.0 * time_factor
        expected = (current + increase) if current else None

        warnings = []
        if dose > 4000:
            warnings.append(f"UL 초과: {dose} > 4000 IU/day")

        return SimulationResult(
            parameter="Vitamin D",
            current_value=current,
            expected_value=expected,
            expected_change=increase,
            interpretation=f"비타민 D {dose} IU/day, {weeks}주 → +{increase:.1f} ng/mL",
            warnings=warnings,
            monitoring_recommendations=["3개월 후 25(OH)D", "칼슘 모니터링"]
        )

    def _simulate_calcium(self, patient, intervention):
        """칼슘 → 골절 위험"""
        baseline_risk = 0.20 if patient.fracture_risk_high else 0.10

        if intervention.vitamin_d_iu and intervention.vitamin_d_iu >= 800:
            reduction = 0.15
            expected = baseline_risk * (1 - reduction)
            interp = f"칼슘 {intervention.calcium_mg}mg + VitD {intervention.vitamin_d_iu} IU → 골절 위험 -15%"
        else:
            expected = baseline_risk
            interp = f"칼슘 {intervention.calcium_mg}mg (VitD 병용 권장)"

        return SimulationResult(
            parameter="Fracture Risk",
            current_value=baseline_risk,
            expected_value=expected,
            expected_change=-(baseline_risk - expected),
            interpretation=interp,
            monitoring_recommendations=["DEXA 골밀도", "낙상 위험 평가"]
        )

    def _simulate_omega3(self, patient, intervention):
        """오메가-3 → CVD 위험"""
        baseline = 0.15 if patient.age >= 70 else 0.10
        dose = intervention.omega3_epa_dha_g

        if dose >= 1.0:
            reduction = 0.08
            expected = baseline * (1 - reduction)
            interp = f"오메가-3 {dose}g/day → CVD 위험 -8%"
        else:
            expected = baseline
            interp = f"오메가-3 {dose}g/day (1.0g 이상 권장)"

        return SimulationResult(
            parameter="CVD Risk",
            current_value=baseline,
            expected_value=expected,
            expected_change=-(baseline - expected),
            interpretation=interp,
            monitoring_recommendations=["지질 프로필", "출혈 경향 (항응고제 복용 시)"]
        )

    def _simulate_vitamin_c(self, patient, intervention):
        """✅ 비타민 C 섭취 적정성 평가"""
        gl = self.guidelines['vitamin_c']
        dose = intervention.vitamin_c_mg
        warnings = []
        contraindications = []
        monitoring = []

        # 용량 수준 판정
        if dose < gl['rni']:
            interp = f'{dose}mg/day - 권장섭취량 미달 (결핍 위험)'
        elif dose < gl['optimal_range'][0]:
            interp = f'{dose}mg/day - 권장섭취량 충족'
        elif dose <= gl['optimal_range'][1]:
            interp = f'{dose}mg/day - 최적 범위 (항산화, 면역 지원)'
        elif dose <= gl['upper_limit']:
            interp = f'{dose}mg/day - 고용량이나 안전 범위'
            warnings.append('1000mg↑ 시 분할 복용 권장')
        else:
            interp = f'{dose}mg/day - UL 초과 (설사, 위장 불편)'
            warnings.append(f'UL {gl["upper_limit"]}mg 초과')

        # 특수 집단
        if patient.smoker and dose < 150:
            warnings.append('흡연자 → +50-100mg 권장 (150-200mg/day)')

        if patient.immune_compromised and dose < 200:
            warnings.append('면역저하 → 200-500mg/day 권장')

        # 신결석 위험 (남성)
        if patient.sex == 'M' and dose >= gl['kidney_stone_risk_threshold_male']:
            if patient.kidney_stone_history:
                contraindications.append('신결석 병력 남성: ≥1000mg 금기 (위험 2배↑)')
            else:
                warnings.append('남성 ≥1000mg: 신결석 위험↑')

        # 철분 과잉
        if patient.hemochromatosis:
            contraindications.append('혈색소침착증: VitC가 철분 흡수↑')

        # 철결핍 시 유익
        if patient.ferritin and patient.ferritin < 30:
            monitoring.append('철결핍 빈혈: VitC+철분 병용 시 흡수 67%↑')

        return SimulationResult(
            parameter='Vitamin C Status',
            current_value=None,
            expected_value=None,
            expected_change=None,
            interpretation=interp,
            warnings=warnings,
            contraindications=contraindications,
            monitoring_recommendations=monitoring
        )

    def generate_report(self, patient, intervention, results):
        """리포트 생성"""
        lines = []
        lines.append("\n" + "="*80)
        lines.append("🏥 영양 중재 시뮬레이션 결과")
        lines.append("="*80)
        lines.append(f"\n환자: {patient.age}세 {patient.sex}, 기간: {intervention.duration_weeks}주\n")

        for name, res in results.items():
            lines.append("─"*80)
            lines.append(f"📊 {res.parameter}")
            lines.append("─"*80)
            lines.append(f"   [{res.model_type.upper()}]")

            if res.current_value is not None:
                lines.append(f"   현재: {res.current_value:.2f}")
            if res.expected_value is not None:
                lines.append(f"   예상: {res.expected_value:.2f}")
            if res.expected_change is not None:
                sign = "+" if res.expected_change > 0 else ""
                lines.append(f"   변화: {sign}{res.expected_change:.2f}")

            lines.append(f"\n   {res.interpretation}")

            if res.warnings:
                lines.append("\n   ⚠️  주의:")
                for w in res.warnings:
                    lines.append(f"      • {w}")

            if res.contraindications:
                lines.append("\n   🚫 금기:")
                for c in res.contraindications:
                    lines.append(f"      • {c}")

            if res.monitoring_recommendations:
                lines.append("\n   📌 모니터링:")
                for m in res.monitoring_recommendations:
                    lines.append(f"      • {m}")

            lines.append("")

        lines.append("="*80)
        lines.append("⚠️  연구/교육용 - 실제 임상 결정에 사용 불가")
        lines.append("="*80)

        return "\n".join(lines)

print("✅ IntegratedNutritionSimulator 정의 완료 (Vitamin C 포함!)")

✅ IntegratedNutritionSimulator 정의 완료 (Vitamin C 포함!)


## 6️⃣ 모델 로드

In [ ]:
# 모델 경로 설정
MODEL_PATH = '/content/drive/MyDrive/ColabNotebooks/albumin_predictor_improved.pkl'

try:
    albumin_model = AlbuminMLPredictor(MODEL_PATH)
    use_ml = True
except:
    print("⚠️  ML 모델 미사용 (Rule-based만)")
    albumin_model = None
    use_ml = False

# 시뮬레이터 초기화
simulator = IntegratedNutritionSimulator(albumin_ml_model=albumin_model)

print("\n" + "="*80)
print("✅ 시뮬레이터 준비 완료")
print("="*80)

✅ ML 모델 로드: RandomForest
   Test R²: 0.3335
🤖 ML + Rule-based 통합 모드

✅ 시뮬레이터 준비 완료


## 7️⃣ 예시: Vitamin C 포함 종합 영양 중재

In [ ]:
# 흡연 남성 + 신결석 병력 + 비타민 C 고용량
patient_vitc = PatientProfile(
    age=68,
    sex='M',
    smoker=True,
    kidney_stone_history=True,
    ferritin=25.0  # 철결핍
)

intervention_vitc = NutritionIntervention(
    vitamin_c_mg=1500,  # 고용량
    duration_weeks=12
)

print("\n🔄 Vitamin C 시뮬레이션...\n")
results_vitc = simulator.simulate(patient_vitc, intervention_vitc)
report_vitc = simulator.generate_report(patient_vitc, intervention_vitc, results_vitc)
print(report_vitc)


🔄 Vitamin C 시뮬레이션...


🏥 영양 중재 시뮬레이션 결과

환자: 68세 M, 기간: 12주

────────────────────────────────────────────────────────────────────────────────
📊 Vitamin C Status
────────────────────────────────────────────────────────────────────────────────
   [RULE-BASED]

   1500mg/day - 고용량이나 안전 범위

   ⚠️  주의:
      • 1000mg↑ 시 분할 복용 권장

   🚫 금기:
      • 신결석 병력 남성: ≥1000mg 금기 (위험 2배↑)

   📌 모니터링:
      • 철결핍 빈혈: VitC+철분 병용 시 흡수 67%↑

⚠️  연구/교육용 - 실제 임상 결정에 사용 불가


## 8️⃣ 예시: 노인 환자 통합 영양 중재 (전체)

In [ ]:
patient_complete = PatientProfile(
    age=82,
    sex='F',

    # Lab
    hemoglobin=11.0,
    ferritin=25.0,
    vitamin_d=15.0,
    albumin=3.2,

    # ML용
    bun=28.0,
    creatinine=1.3,
    glucose=95.0,
    sodium=139.0,
    potassium=4.2,
    chloride=104.0,
    bicarbonate=24.0,
    wbc=7.5,
    platelet=240.0,

    fracture_risk_high=True,
    smoker=False,
    immune_compromised=True  # 면역저하
)

intervention_complete = NutritionIntervention(
    iron_mg=120,
    vitamin_d_iu=1500,
    calcium_mg=1000,
    omega3_epa_dha_g=1.2,
    vitamin_c_mg=300,  # ✅ 면역저하 → 최적 범위
    protein_g=50,
    duration_weeks=12
)

print("\n🔄 종합 시뮬레이션 (6개 영양소)...\n")
results_complete = simulator.simulate(patient_complete, intervention_complete)
report_complete = simulator.generate_report(patient_complete, intervention_complete, results_complete)
print(report_complete)


🔄 종합 시뮬레이션 (6개 영양소)...


🏥 영양 중재 시뮬레이션 결과

환자: 82세 F, 기간: 12주

────────────────────────────────────────────────────────────────────────────────
📊 Albumin
────────────────────────────────────────────────────────────────────────────────
   [ML]
   현재: 3.20

   ML 예측 실패: 'AGE'

────────────────────────────────────────────────────────────────────────────────
📊 Hemoglobin
────────────────────────────────────────────────────────────────────────────────
   [RULE-BASED]
   현재: 11.00
   예상: 14.60
   변화: +3.60

   철분 120mg/day, 12주 → Hgb +3.6 g/dL

   📌 모니터링:
      • 4주 후 CBC
      • Ferritin/TSAT 추적

────────────────────────────────────────────────────────────────────────────────
📊 Vitamin D
────────────────────────────────────────────────────────────────────────────────
   [RULE-BASED]
   현재: 15.00
   예상: 25.50
   변화: +10.50

   비타민 D 1500 IU/day, 12주 → +10.5 ng/mL

   📌 모니터링:
      • 3개월 후 25(OH)D
      • 칼슘 모니터링

──────────────────────────────────────────────────────────────────────────────

## 9️⃣ 커스텀 시뮬레이션

In [ ]:
# 여기에 원하는 환자 데이터 입력
my_patient = PatientProfile(
    age=70,
    sex='M',
    hemoglobin=11.5,
    vitamin_d=18.0,
    smoker=True,  # ✅ Vitamin C 권장량↑
)

my_intervention = NutritionIntervention(
    iron_mg=100,
    vitamin_d_iu=2000,
    vitamin_c_mg=200,  # ✅ 흡연자 권장
    duration_weeks=8
)

my_results = simulator.simulate(my_patient, my_intervention)
my_report = simulator.generate_report(my_patient, my_intervention, my_results)
print(my_report)


🏥 영양 중재 시뮬레이션 결과

환자: 70세 M, 기간: 8주

────────────────────────────────────────────────────────────────────────────────
📊 Hemoglobin
────────────────────────────────────────────────────────────────────────────────
   [RULE-BASED]
   현재: 11.50
   예상: 13.50
   변화: +2.00

   철분 100mg/day, 8주 → Hgb +2.0 g/dL

   📌 모니터링:
      • 4주 후 CBC
      • Ferritin/TSAT 추적

────────────────────────────────────────────────────────────────────────────────
📊 Vitamin D
────────────────────────────────────────────────────────────────────────────────
   [RULE-BASED]
   현재: 18.00
   예상: 27.33
   변화: +9.33

   비타민 D 2000 IU/day, 8주 → +9.3 ng/mL

   📌 모니터링:
      • 3개월 후 25(OH)D
      • 칼슘 모니터링

────────────────────────────────────────────────────────────────────────────────
📊 Vitamin C Status
────────────────────────────────────────────────────────────────────────────────
   [RULE-BASED]

   200mg/day - 최적 범위 (항산화, 면역 지원)

⚠️  연구/교육용 - 실제 임상 결정에 사용 불가


In [ ]:
import joblib

# 1. Define the SAVE_PATH variable
SAVE_PATH = '/content/drive/MyDrive/ColabNotebooks/integrated_nutrition_simulator.pkl'

print("\n" + "="*80)
print(f"💾 시뮬레이터 저장 경로: {SAVE_PATH}")
print("="*80)

# 2. Use joblib.dump() to save the simulator object
# 3. Include a try-except block to handle potential errors
try:
    joblib.dump(simulator, SAVE_PATH)
    print("\n✅ IntegratedNutritionSimulator 객체가 성공적으로 저장되었습니다.")
except Exception as e:
    print(f"\n❌ IntegratedNutritionSimulator 객체 저장 중 오류 발생: {e}")

print("="*80)


💾 시뮬레이터 저장 경로: /content/drive/MyDrive/ColabNotebooks/integrated_nutrition_simulator.pkl

✅ IntegratedNutritionSimulator 객체가 성공적으로 저장되었습니다.
